[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mengyulin/CoastalEngineering/blob/master/Showcase/Chap_3/2_DispersionEq.ipynb)

## 載入 Python 函式庫

In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

# 使用 Newton Raphson 法求解頻散關係式 (最精確解)

source: https://github.com/pyoceans/python-oceans

頻散關係式

$$
\omega^2 = g k \tanh kh
$$

使用方式 (詳見程式內註解)：
1. 先載入 `class Waves` (在同個 Jupyter Notebook 中載入一次即可)。
2. 呼叫 `Waves` 並輸入水深 `h` 與週期 `T`，即可輸出波長、波速、群速等值。
3. 亦可輸入水深 `h`、波長 `L` 以計算週期。

In [2]:
class Waves:
    r"""
    Solves the wave dispersion relationship via Newton-Raphson.

    .. math::
        \omega^2 = gk\tanh kh

    Parameters
    ----------
    h : array_like, str
        Water depth [m] or 'deep', 'shallow' as keywords
    T : array_like
        Wave period [s]
    L : array_like
        Wave length [m]
    thetao : array_like
        TODO
    Ho : array_like
        TODO

    Returns
    -------
    omega : array_like
            Wave frequency
    TODO: hoLo, hoL, Lo, L, k, T, Co, C, Cg, G, Ks, Kr, theta, H

    Notes
    -----
    Compare values with:
    http://www.coastal.udel.edu/faculty/rad/wavetheory.html

    Examples
    --------
    >>> from oceans.sw_extras.waves import Waves
    >>> wav = Waves(h=10, T=5, L=None)
    >>> print(f"ho/Lo = {wav.hoLo:.3f}")
    ho/Lo = 0.256
    >>> print(f"ho/L  = {wav.hoL:.3f}")
    ho/L  = 0.273
    >>> print(f"Lo    = {wav.Lo:.3f}")
    Lo    = 39.033
    >>> print(f"L     = {wav.L:.3f}")
    L     = 36.593
    >>> print(f"k     = {wav.k:.3f}")
    k     = 0.172
    >>> print(f"omega = {wav.omega:.3f}")
    omega = 1.257
    >>> print(f"T     = {wav.T:.3f}")
    T     = 5.000
    >>> print(f"C     = {wav.C:.3f}")
    C     = 7.319
    >>> print(f"Cg    = {wav.Cg:.3f}")
    Cg    = 4.471
    >>> print(f"G     = {wav.G:.3f}")
    G     = 0.222
    >>> wav = Waves(h=10, T=None, L=100)
    >>> print(f"ho/Lo = {wav.hoLo:.3f}")
    ho/Lo = 0.056
    >>> print(f"ho/L  = {wav.hoL:.3f}")
    ho/L  = 0.100
    >>> print(f"Lo    = {wav.Lo:.3f}")
    Lo    = 179.568
    >>> print(f"L     = {wav.L:.3f}")
    L     = 100.000
    >>> print(f"k     = {wav.k:.3f}")
    k     = 0.063
    >>> print(f"omega = {wav.omega:.3f}")
    omega = 0.586
    >>> print(f"T     = {wav.T:.3f}")
    T     = 10.724
    >>> print(f"C     = {wav.C:.3f}")
    C     = 9.325
    >>> print(f"Cg    = {wav.Cg:.3f}")
    Cg    = 8.291
    >>> print(f"G     = {wav.G:.3f}")
    G     = 0.778
    >>> print(f"Ks  = {wav.Ks:.3f}")
    Ks  = 1.005

    """

    def __init__(self, h, T=None, L=None, thetao=None, Ho=None, lat=None):
        self.T = np.asarray(T, dtype=np.float64)
        self.L = np.asarray(L, dtype=np.float64)
        self.Ho = np.asarray(Ho, dtype=np.float64)
        self.lat = np.asarray(lat, dtype=np.float64)
        self.thetao = np.asarray(thetao, dtype=np.float64)

        if isinstance(h, str):
            if L is not None:
                if h == "deep":
                    self.h = self.L / 2.0
                elif h == "shallow":
                    self.h = self.L * 0.05
        else:
            self.h = np.asarray(h, dtype=np.float64)

        g = 9.81  # Default gravity.

        if L is None:
            self.omega = 2 * np.pi / self.T
            self.Lo = (g * self.T**2) / 2 / np.pi
            # Returns wavenumber of the gravity wave dispersion relation using
            # newtons method. The initial guess is shallow water wavenumber.
            self.k = self.omega / np.sqrt(g)
            # TODO: May change to,
            # self.k = self.w ** 2 / (g * np.sqrt(self.w ** 2 * self.h / g))
            f = g * self.k * np.tanh(self.k * self.h) - self.omega**2

            while np.abs(f.max()) > 1e-10:
                dfdk = g * self.k * self.h * (
                    1 / (np.cosh(self.k * self.h))
                ) ** 2 + g * np.tanh(self.k * self.h)
                self.k = self.k - f / dfdk
                # FIXME:
                f = g * self.k * np.tanh(self.k * self.h) - self.omega**2

            self.L = 2 * np.pi / self.k
            if isinstance(h, str):
                if h == "deep":
                    self.h = self.L / 2.0
                elif h == "shallow":
                    self.h = self.L * 0.05
        else:
            self.Lo = self.L / np.tanh(2 * np.pi * self.h / self.L)
            self.k = 2 * np.pi / self.L
            self.T = np.sqrt(2 * np.pi * self.Lo / g)
            self.omega = 2 * np.pi / self.T

        self.hoL = self.h / self.L
        self.hoLo = self.h / self.Lo
        self.C = self.omega / self.k  # or L / T
        self.Co = self.Lo / self.T
        self.G = 2 * self.k * self.h / np.sinh(2 * self.k * self.h)
        self.n = (1 + self.G) / 2
        self.Cg = self.n * self.C
        self.Ks = np.sqrt(1 / (1 + self.G) / np.tanh(self.k * self.h))

        if thetao is None:
            self.theta = np.NaN
            self.Kr = np.NaN
        if thetao is not None:
            self.theta = np.rad2deg(
                np.asin(self.C / self.Co * np.sin(np.deg2rad(self.thetao))),
            )
            self.Kr = np.sqrt(
                np.cos(np.deg2rad(self.thetao)) / np.cos(np.deg2rad(self.theta)),
            )

        if Ho is None:
            self.H = np.NaN
        if Ho is not None:
            self.H = self.Ho * self.Ks * self.Kr

### 計算一
計算 h = 0.5 m, b = 0.1 m, 不同 kb 值之波長 L 與週期 T:

In [32]:
b = 0.1
h = 0.5
kb = np.linspace(0.3, 1.2, 10)
ka = np.array([0.05, 0.1])
wave_number_list = kb/b

# 印出 org-mode table 標題
print("| No. |  kb  |   ka  |  H (m)  |  L (m)  |  T (s) |")
print("|-----+------+-------+---------+---------+--------|")

for i, k in enumerate(wave_number_list, start=1):
    wav = Waves(h, T=None, L=2 * np.pi / k)
    print(f"| {i:3d} | {kb[i-1]:4.2f} | {ka[0]:5.2f} | {ka[0]/k*2:7.4f} | {wav.L:7.4f} | {wav.T:6.3f} |")
for i, k in enumerate(wave_number_list, start=1):
    wav = Waves(h, T=None, L=2 * np.pi / k)
    print(f"| {i+10:3d} | {kb[i-1]:4.2f} | {ka[1]:5.2f} | {ka[1]/k*2:7.4f} | {wav.L:7.4f} | {wav.T:6.3f} |")

| No. |  kb  |   ka  |  H (m)  |  L (m)  |  T (s) |
|-----+------+-------+---------+---------+--------|
|   1 | 0.30 |  0.05 |  0.0333 |  2.0944 |  1.217 |
|   2 | 0.40 |  0.05 |  0.0250 |  1.5708 |  1.022 |
|   3 | 0.50 |  0.05 |  0.0200 |  1.2566 |  0.903 |
|   4 | 0.60 |  0.05 |  0.0167 |  1.0472 |  0.821 |
|   5 | 0.70 |  0.05 |  0.0143 |  0.8976 |  0.759 |
|   6 | 0.80 |  0.05 |  0.0125 |  0.7854 |  0.709 |
|   7 | 0.90 |  0.05 |  0.0111 |  0.6981 |  0.669 |
|   8 | 1.00 |  0.05 |  0.0100 |  0.6283 |  0.634 |
|   9 | 1.10 |  0.05 |  0.0091 |  0.5712 |  0.605 |
|  10 | 1.20 |  0.05 |  0.0083 |  0.5236 |  0.579 |
|  11 | 0.30 |  0.10 |  0.0667 |  2.0944 |  1.217 |
|  12 | 0.40 |  0.10 |  0.0500 |  1.5708 |  1.022 |
|  13 | 0.50 |  0.10 |  0.0400 |  1.2566 |  0.903 |
|  14 | 0.60 |  0.10 |  0.0333 |  1.0472 |  0.821 |
|  15 | 0.70 |  0.10 |  0.0286 |  0.8976 |  0.759 |
|  16 | 0.80 |  0.10 |  0.0250 |  0.7854 |  0.709 |
|  17 | 0.90 |  0.10 |  0.0222 |  0.6981 |  0.669 |
|  18 | 1.00